In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import folium 
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/seoul-apt-price-withloc/merged_dataset_2019_2020.csv


### Intro
* In this note, I create interactive maps using a data set that includes the prices of seoul apartments.
> * These maps show the distribution of APT complexes whose transaction volumes are greater than certain levels and their average prices.
* The package used: `panda` and `folium`

### Import data

* We use a data set that includes transaction prices of seoul apartments from Dec 2019 to Dec 2020 and their location information (latitude and longitude). 


In [2]:
# import data
dat = pd.read_csv("../input/seoul-apt-price-withloc/merged_dataset_2019_2020.csv")
dat.head()

,address,size,year,month,price,floor,const_year,lat,lng
0,서울특별시 강남구 개포동\t655-2\t0655\t0002\t개포2차현대아파트(220),77.75,2020,6,"149,500",4,1988,37.482630,127.051104
1,서울특별시 강남구 개포동\t658-1\t0658\t0001\t개포6차우성아파트1동~8동,54.98,2020,2,"144,000",4,1987,37.476658,127.056909
2,서울특별시 강남구 개포동\t658-1\t0658\t0001\t개포6차우성아파트1동~8동,79.97,2020,6,"162,750",4,1987,37.476658,127.056909
3,서울특별시 강남구 개포동\t658-1\t0658\t0001\t개포6차우성아파트1동~8동,79.97,2020,6,"160,000",5,1987,37.476658,127.056909
4,서울특별시 강남구 개포동\t658-1\t0658\t0001\t개포6차우성아파트1동~8동,79.97,2020,6,"150,000",2,1987,37.476658,127.056909


### Process data


1. Define the type of `price` column properly.
2. Calculate average price for each APT complex, year and month.
3. Calculate transaction volume for each APT complex and reference periods (for last 3months/6months/1year).

In [3]:
# change the type of price from object to float64
for line in range(0, len(dat)):
    dat["price"][line]= float(dat["price"][line].replace(",",""))

dat["price"] = dat["price"].astype("float64")
dat.dtypes

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


address        object
size          float64
year            int64
month           int64
price         float64
floor           int64
const_year      int64
lat           float64
lng           float64
dtype: object

In [4]:
dat.head()

,address,size,year,month,price,floor,const_year,lat,lng
0,서울특별시 강남구 개포동\t655-2\t0655\t0002\t개포2차현대아파트(220),77.75,2020,6,149500.0,4,1988,37.482630,127.051104
1,서울특별시 강남구 개포동\t658-1\t0658\t0001\t개포6차우성아파트1동~8동,54.98,2020,2,144000.0,4,1987,37.476658,127.056909
2,서울특별시 강남구 개포동\t658-1\t0658\t0001\t개포6차우성아파트1동~8동,79.97,2020,6,162750.0,4,1987,37.476658,127.056909
3,서울특별시 강남구 개포동\t658-1\t0658\t0001\t개포6차우성아파트1동~8동,79.97,2020,6,160000.0,5,1987,37.476658,127.056909
4,서울특별시 강남구 개포동\t658-1\t0658\t0001\t개포6차우성아파트1동~8동,79.97,2020,6,150000.0,2,1987,37.476658,127.056909


In [5]:
# create yearmon column: year + month
def create_yearmon(row):
    x= str(row["year"])+"-"+ str(row["month"])
    return x
dat['yearmon'] = dat.apply(lambda row: create_yearmon(row), axis = 1)
dat.head()

,address,size,year,month,price,floor,const_year,lat,lng,yearmon
0,서울특별시 강남구 개포동\t655-2\t0655\t0002\t개포2차현대아파트(220),77.75,2020,6,149500.0,4,1988,37.482630,127.051104,2020-6
1,서울특별시 강남구 개포동\t658-1\t0658\t0001\t개포6차우성아파트1동~8동,54.98,2020,2,144000.0,4,1987,37.476658,127.056909,2020-2
2,서울특별시 강남구 개포동\t658-1\t0658\t0001\t개포6차우성아파트1동~8동,79.97,2020,6,162750.0,4,1987,37.476658,127.056909,2020-6
3,서울특별시 강남구 개포동\t658-1\t0658\t0001\t개포6차우성아파트1동~8동,79.97,2020,6,160000.0,5,1987,37.476658,127.056909,2020-6
4,서울특별시 강남구 개포동\t658-1\t0658\t0001\t개포6차우성아파트1동~8동,79.97,2020,6,150000.0,2,1987,37.476658,127.056909,2020-6


In [6]:
# calculate transaction volume for each apt complex and yearmon
dat["avgprice"] = dat.groupby(["address", "yearmon"])["price"].transform("mean")

In [7]:
# calculate transaction volume for each apt complex and periods (3months/6months/1year)
def count_past_sales(series, intval = "90D"):
    series2 = pd.to_datetime(series)
    sale_time = pd.Series(series2.index, index=series2, name='count_sales_this_month').sort_index() # exchange the positions of index and values
    count_sales = sale_time.rolling(intval, min_periods=1).count()
    count_sales = count_sales.groupby(count_sales.index.month).transform('last')
    return count_sales

# transaction volume for last 3 months
aa=dat.groupby('address')["yearmon"].apply(count_past_sales)
bb=dat.groupby('address')["yearmon"].apply(lambda series: count_past_sales(series, intval = "180D"))
cc=dat.groupby('address')["yearmon"].apply(lambda series: count_past_sales(series, intval = "365D"))

# put index
aa.index=dat.sort_values(by=['address','year', "month"]).index
bb.index=dat.sort_values(by=['address','year', "month"]).index
cc.index=dat.sort_values(by=['address','year', "month"]).index

dat['num_sales_last_3months']=aa
dat['num_sales_last_6months']=bb
dat['num_sales_last_12months']=cc

In [8]:
# drop observations with inappropriate location info
dat.drop(dat[dat["lng"].isnull() == True].index,axis = 0, inplace = True)
dat.drop(dat[dat["lat"].isnull() == True].index,axis = 0, inplace = True)

In [9]:
# create a column: "color" using price information
def colouring(row):
    if (row["avgprice"] > 0) and (row["avgprice"] <= 50000): # less than $500k 
        return "yellow"
    elif (row["avgprice"] > 50000) and (row["avgprice"] <= 100000): # more than $500k and less than $1m
        return "orange"
    elif (row["avgprice"] > 100000) and (row["avgprice"] <= 150000): # more than $1m and less than $1.5m
        return "red"
    elif (row["avgprice"] > 150000) and (row["avgprice"] <= 200000): # more than $1.5m and less than $2m
        return "darkred"
    return "darkpurple"
dat["color"] = dat.apply(colouring, axis = 1)
dat.head()

,address,size,year,month,price,floor,const_year,lat,lng,yearmon,avgprice,num_sales_last_3months,num_sales_last_6months,num_sales_last_12months,color
0,서울특별시 강남구 개포동\t655-2\t0655\t0002\t개포2차현대아파트(220),77.75,2020,6,149500.0,4,1988,37.482630,127.051104,2020-6,149500.0,1.0,1.0,1.0,red
1,서울특별시 강남구 개포동\t658-1\t0658\t0001\t개포6차우성아파트1동~8동,54.98,2020,2,144000.0,4,1987,37.476658,127.056909,2020-2,144000.0,1.0,1.0,1.0,red
2,서울특별시 강남구 개포동\t658-1\t0658\t0001\t개포6차우성아파트1동~8동,79.97,2020,6,162750.0,4,1987,37.476658,127.056909,2020-6,151550.0,5.0,6.0,6.0,darkred
3,서울특별시 강남구 개포동\t658-1\t0658\t0001\t개포6차우성아파트1동~8동,79.97,2020,6,160000.0,5,1987,37.476658,127.056909,2020-6,151550.0,5.0,6.0,6.0,darkred
4,서울특별시 강남구 개포동\t658-1\t0658\t0001\t개포6차우성아파트1동~8동,79.97,2020,6,150000.0,2,1987,37.476658,127.056909,2020-6,151550.0,5.0,6.0,6.0,darkred


### Default setting for an interactive map

In [10]:
# define a map
my_map = folium.Map(
    location = [37.544294, 127.0016985], 
    width="%40",
    height="%35",
    zoom_start= 12)


### Create interactive maps

In [11]:
# Function for making a subdat set
def mapping(nums = "num_sales_last_3months", min_transvol = 10, yr_start = 2020, yr_end = 2020):
    subdat = dat[(dat[nums]>min_transvol) & (dat["year"] >= yr_start) & (dat["year"] <= yr_end)]
    subdat = subdat.drop_duplicates(subset=['address','yearmon'])
    for _, apt in subdat.iterrows():
        folium.Marker(
        location = [apt["lat"], apt["lng"]],
        popup = apt["address"],
        tooltip = apt[nums],
        icon = folium.Icon(color = apt["color"])).add_to(my_map)
    return(my_map)

* The interactive map below shows all apartment complexes whose transaction voloume were more than 50 for `last 3 months` as of 2020
* The colour of icons shows the level of average prices of the apartment complexes.
 
 > 1. yellow: less than $\$500k$
 > 2. red:  from $\$500k$ to $\$1m$
 > 3. darkred:  from $\$1m$ to $\$1.5m$
 > 4. darkpurple: more than $\$1.5m$
  

In [12]:
mapping(min_transvol =  50)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:10: UserWarning: color argument of Icon should be one of: {'darkpurple', 'orange', 'lightgray', 'red', 'black', 'darkgreen', 'darkblue', 'gray', 'pink', 'lightgreen', 'lightred', 'blue', 'green', 'purple', 'lightblue', 'darkred', 'cadetblue', 'white', 'beige'}.
  # Remove the CWD from sys.path while we load stuff.


* The interactive map below shows all apartment complexes whose transaction voloume were more than 50 for `last 6 months` as of 2020


In [13]:
mapping(nums="num_sales_last_6months", min_transvol =  50)


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:10: UserWarning: color argument of Icon should be one of: {'darkpurple', 'orange', 'lightgray', 'red', 'black', 'darkgreen', 'darkblue', 'gray', 'pink', 'lightgreen', 'lightred', 'blue', 'green', 'purple', 'lightblue', 'darkred', 'cadetblue', 'white', 'beige'}.
  # Remove the CWD from sys.path while we load stuff.
